# Data: Past, Present, Future
# Lab 12b: Fairness, accountability, transparency
    

# Fairness


## Two major doctrines

### Disparate treatment

> one kind of unlawful discrimination in US labor law. In the United States, it means unequal behavior toward someone because of a protected characteristic (e.g. race or gender) under Title VII of the United States Civil Rights Act.  (h/t wikipedia)

Disparate treatment is a form of *procedural* justice, concerned with process. 

- discrimination built into a process

    - formal: explicitly using class membership, OR

    - intentional: *purposefully* attempting to discriminate without direct reference to class membership
        - use zipcodes rather than race *deliberately* to exclude from your country club
    

- equality of opportunity

## Disparate impact 

>in United States labor law refers to practices in employment, housing, and other areas that adversely affect one group of people of a protected characteristic more than another, even though rules applied by employers or landlords are formally neutral. Although the protected classes vary by statute, most federal civil rights laws protect based on race, color, religion, national origin, and sex as protected traits, and some laws include disability status and other traits as well. (h/t wikipedia)

Disparate impact is a form of *distributive* justice, concerned with the outcomes of processes.

- equality of outcome

- substantial justice



Let's look at a much studied example: the "adult" data set. Task is to predict whether person makes >$50K, to give credit or the like.
(https://archive.ics.uci.edu/ml/datasets/adult)

Paper examining: 

> Michael Feldman, Sorelle A. Friedler, John Moeller, Carlos Scheidegger, and Suresh Venkatasubramanian. Certifying and Removing Disparate Impact. Proceedings of the 21st ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, 2015. (http://arxiv.org/abs/1412.3756)

Their awesome code: (https://github.com/algofairness/fairness-comparison/tree/master/algorithms)

Kaggle competition (https://www.kaggle.com/uciml/adult-census-income) 


In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
names = ['age','workclass', 'fnlwgt', 'education', 'education.num', 'marital.status','occupation', 'relationship','race','sex','capital.gain','capital.loss', 'hours.per.week', 'native.country','income']

In [ ]:
data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', sep=',', header=None, na_values="?", names=names)

In [ ]:
data.head(20)

In [ ]:
data.groupby(["race","income"]).size()

In [ ]:
data.groupby(["sex", "income"]).size()

In [ ]:
data.groupby(["education", "income"]).size()

### We're only going to want to train on some of the features.

In [ ]:
features = ['age','workclass','education','marital.status','occupation','education.num','race','sex','relationship','capital.gain','capital.loss','native.country','income']

In [ ]:
## converts strings to numerical values
## from https://www.kaggle.com/dewilliams/ml-adult-income

from sklearn.preprocessing import LabelEncoder

def preprocess_features(dframe):
    for column in dframe:
        enc = LabelEncoder()
        if(column not in ['age','education.num','fnlwgt','capital.gain','capital.loss','hours.per.week']):
            dframe[column] = enc.fit_transform(dframe[column])
    return dframe

In [ ]:
data=data[features]

In [ ]:
data = preprocess_features(data)

In [ ]:
data.head(20)

In [ ]:
# works better if extract from pandas dataframe
# separate the existing classification (the diagnosis) from the features tested
data_array=data.values
y = data['income']
X = data.drop('income', axis=1)

In [ ]:
#just copying technique we saw in Lab 10

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45, random_state=42)

dt = DecisionTreeClassifier() #set up classifier, with all **default** values
clf=dt.fit(X_train, y_train) #fit on training data

In [ ]:
# display the relative importance of each attribute
relval = clf.feature_importances_

# horizontal bar plot of feature importance
pos = np.arange(12) + 0.5
plt.barh(pos, relval, align='center')
plt.title("Feature Importance")
plt.xlabel("")
plt.ylabel("Features")
plt.yticks(pos, ('Age','Working Class','Education','Marital Status','Occupation','Education Grade','Race','Sex','Relationship Status','Capital Gain','Capital Loss','Native Country'))
plt.grid(True)

# What have we trained the decision tree to to do?


# What sorts of fairness is this likely to fail?

In [ ]:
X_test

In [ ]:
X_test[X_test["sex"]==1]

In [ ]:
clf.predict(X_test).mean()

In [ ]:
clf.predict(X_test[X_test["sex"]==0]).mean()

In [ ]:
clf.predict(X_test[X_test["sex"]==0]).mean()

# Disparate impact operationalized as "80-20 rule"


Look at *ratio* of impact of a procedure. 

If hire 20 women for every 50 men, ratio is 20/50 which equals .4. 

40% < 80%

Therefore, 

hiring practices have disparate impact *even if there is no deliberate discrimination using protected attributes*

**Still can be 20% discriminated against!!**

How does our classifier do?


## What if we rebuilt our classifier without ever looking at legally protected attributes?

### Don't let our classifier use the features race and sex.

In [ ]:
protected_attributes=['race', 'sex']

In [ ]:
X_protected = data.drop(protected_attributes, axis=1)

In [ ]:
#train anew

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_protected, y, test_size=0.45, random_state=42)

dt = DecisionTreeClassifier() #set up classifier, with all **default** values
clf=dt.fit(X_train, y_train) #fit on training data

In [ ]:
clf.predict(X_test).mean()

In [ ]:
clf.predict(X_test[X["sex"]==0]).mean()

In [ ]:
clf.predict(X_test[X["sex"]==1]).mean()

## What sort of *fairness* does this fail?

## What to do?